In [1]:
import pandas as pd 
import numpy as np

In [2]:
DRKG_PATH = "/Users/elifozkan/Desktop/translator-openpredict/data/drkg/embed/drkg.tsv"
drkg_data = pd.read_table(DRKG_PATH, sep = "\t")
#entity2src = pd.read_table("/Users/elifozkan/Desktop/translator-openpredict/data/drkg/entity2src.tsv",sep = "\t",on_bad_lines='skip')
entity2src = pd.read_csv("/Users/elifozkan/Desktop/translator-openpredict/data/drkg/entity2src.csv",on_bad_lines='skip')
entity2src.head()

,Subject,Source1,Source2,Source3,Source4,Source5,Source6
0,Gene::,[IntAct] https://www.ebi.ac.uk/intact/,NaN,NaN,NaN,NaN,NaN
1,Gene::1,[Bibliograpic sources] Collection of datasourc...,[Drugbank] https://www.drugbank.ca/atc,[STRING] https://string-db.org/,[Hetionet] Biomedical knowledge graph https://...,[GNBR] Data extracted from biomedical texts se...,[IntAct] https://www.ebi.ac.uk/intact/
2,Gene::1000,[Hetionet] Biomedical knowledge graph https://...,[STRING] https://string-db.org/,[GNBR] Data extracted from biomedical texts se...,[IntAct] https://www.ebi.ac.uk/intact/,NaN,NaN
3,Gene::10000,[STRING] https://string-db.org/,[DGIdb] http://www.dgidb.org/,[Hetionet] Biomedical knowledge graph https://...,[GNBR] Data extracted from biomedical texts se...,[IntAct] https://www.ebi.ac.uk/intact/,NaN
4,Gene::100000711,[GNBR] Data extracted from biomedical texts se...,NaN,NaN,NaN,NaN,NaN


In [3]:
def split_sources(df):
    split_df = df.copy()
    sources = ["Source1", "Source2", "Source3", "Source4", "Source5", "Source6"]
    for source in sources: 
        split_df[source] = split_df[source].str.split()
    return split_df
        

def get_source_name(split_df):
    source_df = split_df.copy()
    sources = ["Source1", "Source2", "Source3", "Source4", "Source5", "Source6"]
    for source in sources : 
        source_df[source] = source_df[source].str[0]
    return source_df


In [4]:
split_df = split_sources(entity2src)
source_df = get_source_name(split_df)

def split_entity_types(df) : 
    df["SplitSubject"] = df["Subject"].str.split("::")
    
def split_object_types(df): 
     df["SplitObject"] = df["Object"].str.split("::")

split_entity_types(source_df)
source_df.head()

,Subject,Source1,Source2,Source3,Source4,Source5,Source6,SplitSubject
0,Gene::,[IntAct],NaN,NaN,NaN,NaN,NaN,"[Gene, ]"
1,Gene::1,[Bibliograpic,[Drugbank],[STRING],[Hetionet],[GNBR],[IntAct],"[Gene, 1]"
2,Gene::1000,[Hetionet],[STRING],[GNBR],[IntAct],NaN,NaN,"[Gene, 1000]"
3,Gene::10000,[STRING],[DGIdb],[Hetionet],[GNBR],[IntAct],NaN,"[Gene, 10000]"
4,Gene::100000711,[GNBR],NaN,NaN,NaN,NaN,NaN,"[Gene, 100000711]"


In [5]:
split_entity_types(source_df)
source_df.head()

,Subject,Source1,Source2,Source3,Source4,Source5,Source6,SplitSubject
0,Gene::,[IntAct],NaN,NaN,NaN,NaN,NaN,"[Gene, ]"
1,Gene::1,[Bibliograpic,[Drugbank],[STRING],[Hetionet],[GNBR],[IntAct],"[Gene, 1]"
2,Gene::1000,[Hetionet],[STRING],[GNBR],[IntAct],NaN,NaN,"[Gene, 1000]"
3,Gene::10000,[STRING],[DGIdb],[Hetionet],[GNBR],[IntAct],NaN,"[Gene, 10000]"
4,Gene::100000711,[GNBR],NaN,NaN,NaN,NaN,NaN,"[Gene, 100000711]"


In [6]:
def create_entity_type_df(df): 
    entity_type_df = df.copy()
    entity_type_df["SubjectType"] = entity_type_df["SplitSubject"].str[0]
    entity_type_df["SubjectID"] = entity_type_df["SplitSubject"].str[1]
    return entity_type_df

def create_object_type(df) : 
    entity_type_df = df.copy()
    entity_type_df["ObjectType"] = entity_type_df["SplitObject"].str[0]
    entity_type_df["ObjectID"] = entity_type_df["SplitObject"].str[1]
    return entity_type_df

entity_type_df =create_entity_type_df(source_df)
entity_type_df.head()

drkg = pd.merge(drkg_data, entity_type_df, on="Subject")
split_object_types(drkg)
drkg = create_object_type(drkg)
drkg = drkg.drop(["SplitSubject", "SplitObject"],axis = 1)
drkg.head()

,Subject,Predicate,Object,Source1,Source2,Source3,Source4,Source5,Source6,SubjectType,SubjectID,ObjectType,ObjectID
0,Gene::6187,bioarx::HumGenHumGen:Gene:Gene,Gene::5566,[Bibliograpic,[Drugbank],[STRING],[Hetionet],[GNBR],[IntAct],Gene,6187,Gene,5566
1,Gene::6187,bioarx::HumGenHumGen:Gene:Gene,Gene::7412,[Bibliograpic,[Drugbank],[STRING],[Hetionet],[GNBR],[IntAct],Gene,6187,Gene,7412
2,Gene::6187,bioarx::HumGenHumGen:Gene:Gene,Gene::3845,[Bibliograpic,[Drugbank],[STRING],[Hetionet],[GNBR],[IntAct],Gene,6187,Gene,3845
3,Gene::6187,bioarx::HumGenHumGen:Gene:Gene,Gene::3008,[Bibliograpic,[Drugbank],[STRING],[Hetionet],[GNBR],[IntAct],Gene,6187,Gene,3008
4,Gene::6187,bioarx::HumGenHumGen:Gene:Gene,Gene::7083,[Bibliograpic,[Drugbank],[STRING],[Hetionet],[GNBR],[IntAct],Gene,6187,Gene,7083


In [7]:
entity_type_df =create_entity_type_df(source_df)
drkg = pd.merge(drkg_data, entity_type_df, on="Subject")
split_object_types(drkg)
drkg = create_object_type(drkg)
drkg = drkg.drop(["SplitSubject", "SplitObject"],axis = 1)
drkg.head()

,Subject,Predicate,Object,Source1,Source2,Source3,Source4,Source5,Source6,SubjectType,SubjectID,ObjectType,ObjectID
0,Gene::6187,bioarx::HumGenHumGen:Gene:Gene,Gene::5566,[Bibliograpic,[Drugbank],[STRING],[Hetionet],[GNBR],[IntAct],Gene,6187,Gene,5566
1,Gene::6187,bioarx::HumGenHumGen:Gene:Gene,Gene::7412,[Bibliograpic,[Drugbank],[STRING],[Hetionet],[GNBR],[IntAct],Gene,6187,Gene,7412
2,Gene::6187,bioarx::HumGenHumGen:Gene:Gene,Gene::3845,[Bibliograpic,[Drugbank],[STRING],[Hetionet],[GNBR],[IntAct],Gene,6187,Gene,3845
3,Gene::6187,bioarx::HumGenHumGen:Gene:Gene,Gene::3008,[Bibliograpic,[Drugbank],[STRING],[Hetionet],[GNBR],[IntAct],Gene,6187,Gene,3008
4,Gene::6187,bioarx::HumGenHumGen:Gene:Gene,Gene::7083,[Bibliograpic,[Drugbank],[STRING],[Hetionet],[GNBR],[IntAct],Gene,6187,Gene,7083


# Split DRKG into different sources to handle entities more easily
### Drugs are mostly contained in GNBR,Hetionet, Drugbank and DGIdb sources. 
### Diseases are mostly contained in GNBR,Hetionet, Drugbank sources. 
### Gene & Gene interaction info is stores in STRING, IntAct and Hetionet sources. 

In [8]:
Hetionet = drkg.loc[(drkg["Source1"] == "[Hetionet]") | (drkg["Source2"] == "[Hetionet]") | (drkg["Source3"] == "[Hetionet]")
            | (drkg["Source4"] == "[Hetionet]")| (drkg["Source5"] == "[Hetionet]") | (drkg["Source6"] == "[Hetionet]")]

DrugBank = drkg.loc[(drkg["Source1"] == "[Drugbank]") | (drkg["Source2"] == "[Drugbank]") | (drkg["Source3"] == "[Drugbank]")
            | (drkg["Source4"] == "[Drugbank]")| (drkg["Source5"] == "[Drugbank]") | (drkg["Source6"] == "[Drugbank]")]

GNBR = drkg.loc[(drkg["Source1"] == "[GNBR]") | (drkg["Source2"] == "[GNBR]") | (drkg["Source3"] == "[GNBR]")
            | (drkg["Source4"] == "[GNBR]")| (drkg["Source5"] == "[GNBR]") | (drkg["Source6"] == "[GNBR]")]

IntAct = drkg.loc[(drkg["Source1"] == "[IntAct]") | (drkg["Source2"] == "[IntAct]") | (drkg["Source3"] == "[IntAct]")
            | (drkg["Source4"] == "[IntAct]")| (drkg["Source5"] == "[IntAct]") | (drkg["Source6"] == "[IntAct]")]

STRING = drkg.loc[(drkg["Source1"] == "[STRING]") | (drkg["Source2"] == "[STRING]") | (drkg["Source3"] == "[STRING]")
            | (drkg["Source4"] == "[STRING]")| (drkg["Source5"] == "[STRING]") | (drkg["Source6"] == "[STRING]")]

DGIdb = drkg.loc[(drkg["Source1"] == "[DGIdb]") | (drkg["Source2"] == "[DGIdb]") | (drkg["Source3"] == "[DGIdb]")
            | (drkg["Source4"] == "[DGIdb]")| (drkg["Source5"] == "[DGIdb]") | (drkg["Source6"] == "[DGIdb]")]

# Create a file containing drug-disease indications
### Using the predicates indicating treatment from the relation glossary

In [9]:
DrugBank_indication = DrugBank.loc[DrugBank["Predicate"]== "DRUGBANK::treats::Compound:Disease"]
Hetionet_indication = Hetionet.loc[Hetionet["Predicate"] == "Hetionet::CtD::Compound:Disease"]
GNBR_indication = GNBR.loc[(GNBR["Predicate"] == "GNBR::T::Compound:Disease")| (GNBR["Predicate"] == "GNBR::Te::Compound:Disease")]

indications = pd.concat([DrugBank_indication,Hetionet_indication,GNBR_indication])
indications = indications[["SubjectID", "ObjectID"]]
indications.to_csv("/Users/elifozkan/Desktop/translator-openpredict/data/drkg/feature_related/output/drug-disease-indications.csv",index= False)

# Side Effect Similarity from DRKG 
#### The strategy is to get the side effects using the relevant predicates showed in relation glossary and compute the Jaccard similarity scores. 

In [10]:
gnbr_drug_se = GNBR.loc[GNBR["Predicate"] == "GNBR::Sa::Compound:Disease"]
hetionet_drug_se = Hetionet.loc[Hetionet["Predicate"] == "Hetionet::CcSE::Compound:Side Effect"]

In [11]:
mesh_diseases= pd.read_table("/Users/elifozkan/Desktop/translator-openpredict/data/drkg/feature_related/MESH_diseases_vocab.tsv", sep = "\t")
mesh_diseases["d"] = mesh_diseases["d"].str.upper()
mesh_diseases["name"] = mesh_diseases["name"].str.lower()
mesh_diseases = mesh_diseases.rename(columns={"d" : "ObjectID"})

In [12]:
gnbr_drug_se = pd.merge(gnbr_drug_se,mesh_diseases,on = "ObjectID" )

sider_drugs = pd.read_csv("/Users/elifozkan/Desktop/translator-openpredict/data/drkg/feature_related/sider4_vocab.tsv",sep = "\t")
sider_drugs = sider_drugs.rename(columns={"umls_cui_from_meddra" : "ObjectID","side_effect_name": "name"})
sider_drugs["name"] = sider_drugs["name"].str.lower()

hetionet_drug_se = pd.merge(hetionet_drug_se,sider_drugs, on = "ObjectID")
gnbr_drug_se = gnbr_drug_se.loc[gnbr_drug_se["Subject"].str.contains("MESH")==False]

drug_se = pd.concat([gnbr_drug_se,hetionet_drug_se],ignore_index=True)
drug_se = drug_se.drop_duplicates()

In [13]:
a = drug_se["SubjectID"]
b = indications["SubjectID"]
intersect = set(a).intersection(b)
len(intersect)

1985

In [14]:
def se_similarity(df) : 
    import itertools
    drug_se_dict = {k : g["ObjectID"].tolist() for k,g in df.groupby("SubjectID")}
    sim_scores = []

    for combination in itertools.combinations(intersect,2):
        drug1 = combination[0]
        drug2 = combination[1]

        side_effects1 = drug_se_dict[drug1]
        side_effects2 = drug_se_dict[drug2]

        c = set(side_effects1).intersection(side_effects2)
        u = set(side_effects1).union(side_effects2)

        sim_score = len(c) / float(len(u))
        sim_scores.append([drug1,drug2,sim_score])
    drug_se_df = pd.DataFrame(sim_scores, columns = ["Drug1", "Drug2", "SE-SIM"])
    return drug_se_df

In [15]:
se_sim = se_similarity(drug_se)
se_sim.to_csv("/Users/elifozkan/Desktop/translator-openpredict/data/drkg/feature_related/output/drugs-se-sim.csv", index= False)

# Fingeprint Similarity from DRKG
### Drug SMILES representation is available in DRKG.tar file. 

In [16]:
smiles= pd.read_csv("/Users/elifozkan/Desktop/translator-openpredict/data/drkg/feature_related/SMILES.csv",on_bad_lines='skip')
smiles["SMILES"] = smiles["SMILES"].str.replace("\t","")
smiles.head()

,SubjectID,SMILES
0,DB00006,CC[C@H](C)[C@H](NC(=O)[C@H](CCC(O)=O)NC(=O)[C@...
1,DB00007,CCNC(=O)[C@@H]1CCCN1C(=O)[C@H](CCCNC(N)=N)NC(=...
2,DB00014,CC(C)C[C@H](NC(=O)[C@@H](COC(C)(C)C)NC(=O)[C@H...
3,DB00027,CC(C)C[C@@H](NC(=O)CNC(=O)[C@@H](NC=O)C(C)C)C(...
4,DB00035,NC(=O)CC[C@@H]1NC(=O)[C@H](CC2=CC=CC=C2)NC(=O)...


In [17]:
drug_smiles = pd.merge(drug_se, smiles, on = "SubjectID" )
drug_smiles.head()

,Subject,Predicate,Object,Source1,Source2,Source3,Source4,Source5,Source6,SubjectType,SubjectID,ObjectType,ObjectID,name,SMILES
0,Compound::DB00244,GNBR::Sa::Compound:Disease,Disease::MESH:D007674,[Bibliograpic,[Drugbank],[DGIdb],[Hetionet],[GNBR],NaN,Compound,DB00244,Disease,MESH:D007674,kidney diseases,NC1=CC(C(O)=O)=C(O)C=C1
1,Compound::DB00244,GNBR::Sa::Compound:Disease,Disease::MESH:D009395,[Bibliograpic,[Drugbank],[DGIdb],[Hetionet],[GNBR],NaN,Compound,DB00244,Disease,MESH:D009395,"nephritis, interstitial",NC1=CC(C(O)=O)=C(O)C=C1
2,Compound::DB00244,GNBR::Sa::Compound:Disease,Disease::MESH:D010190,[Bibliograpic,[Drugbank],[DGIdb],[Hetionet],[GNBR],NaN,Compound,DB00244,Disease,MESH:D010190,pancreatic neoplasms,NC1=CC(C(O)=O)=C(O)C=C1
3,Compound::DB00244,GNBR::Sa::Compound:Disease,Disease::MESH:D019283,[Bibliograpic,[Drugbank],[DGIdb],[Hetionet],[GNBR],NaN,Compound,DB00244,Disease,MESH:D019283,"pancreatitis, acute necrotizing",NC1=CC(C(O)=O)=C(O)C=C1
4,Compound::DB00244,GNBR::Sa::Compound:Disease,Disease::MESH:D001919,[Bibliograpic,[Drugbank],[DGIdb],[Hetionet],[GNBR],NaN,Compound,DB00244,Disease,MESH:D001919,bradycardia,NC1=CC(C(O)=O)=C(O)C=C1


In [19]:
a = drug_smiles["SubjectID"]
b = drug_se["SubjectID"]
intersect = set(a).intersection(b)
len(intersect)

1854

In [20]:
import oddt
from oddt import toolkit
from oddt import fingerprints

import pandas as pd
import numpy as np
import itertools

input_dict = dict()
for index,line in drug_smiles.iterrows():
     id = line["SubjectID"]

     smiles = line["SMILES"]
     mol = oddt.toolkit.readstring(format="smi",string=smiles)
     fp = mol.calcfp(fptype="MACCS").raw
     input_dict[id] = fp 

In [21]:
def tanimoto_score(fp1, fp2):
    return np.sum(fp1 &  fp2) / np.sum(fp1 | fp2)

In [22]:
c = drug_smiles["SubjectID"]
intersect = set(intersect).intersection(c)

In [23]:
sim_values=[]
for chemical1, chemical2 in itertools.combinations(input_dict.keys(),2):
    TC= tanimoto_score(input_dict[chemical1], input_dict[chemical2])
    if chemical1 != chemical2:
        sim_values.append([chemical1, chemical2, TC])

In [24]:
chem_sim_df = pd.DataFrame(sim_values, columns=['Drug1','Drug2','TC'])
chem_sim_df.head()
chem_sim_df.to_csv("/Users/elifozkan/Desktop/translator-openpredict/data/drkg/feature_related/output/drugs-tc-sim.csv", index=False)

# Target Sequence Similarity from DRKG
### The strategy is to find the target genes using the relevant predicates from DRKG, then match them with their sequence representations taken from hugo database.

In [119]:
def fasta2seq(lines):
    lines = lines[lines.index('\n')+1:]
    lines =lines.replace('\n','')
    return lines

target_sequence = pd.read_csv("/Users/elifozkan/Desktop/translator-openpredict/data/drkg/feature_related/targetseq.csv")
target_sequence.seq =target_sequence.seq.map(fasta2seq)
target_sequence = target_sequence.rename(columns={"geneid": "ObjectID"})
target_sequence["ObjectID"] = target_sequence["ObjectID"].astype("string")
target_sequence.head()

,ObjectID,seq
0,6035,MALEKSLVRLLLLVLILLVLGWVQPSLGKESRAKKFQRQHMDSDSS...
1,635,MPPVGGKKAKKGILERLNAGEIVIGDGGFVFALEKRGYVKAGPWTP...
2,23743,MAPAGRPGAKKGILERLESGEVVIGDGSFLITLEKRGYVKAGLWTP...
3,6301,VLDLDLFRVDKGGDPALIRETQEKRFKDPGLVDQLVKADSEWRRCR...
4,645,AVKKIAIFGATGQTGLTTLAQAVQAGYEVTVLVRDSSRLPSEGPRP...


In [120]:
gnbr_drug_target = GNBR.loc[(GNBR["Predicate"] == "GNBR::B::Compound:Gene") | 
                        (GNBR["Predicate"] == "GNBR::E+::Compound:Gene") |
                        (GNBR["Predicate"] == "GNBR::E-::Compound:Gene") | 
                         (GNBR["Predicate"] == "GNBR::E::Compound:Gene") |
                          (GNBR["Predicate"] == "GNBR::N::Compound:Gene") |
                           (GNBR["Predicate"] == "GNBR::O::Compound:Gene") ]

hetionet_drug_target = Hetionet.loc[(Hetionet["Predicate"] == "Hetionet::CbG::Compound:Gene" )|
                                (Hetionet["Predicate"] == "Hetionet::CdG::Compound:Gene")|
                                (Hetionet["Predicate"] == "Hetionet::CuG::Compound:Gene")]

drugbank_drug_target = DrugBank.loc[DrugBank["Predicate"] == "DRUGBANK::target::Compound:Gene"]



In [121]:
drug_target_seq = pd.concat([hetionet_drug_target,gnbr_drug_target,drugbank_drug_target],ignore_index=True)

In [122]:
drug_target_sequence = pd.merge(target_sequence,drug_target_seq, on = "ObjectID")
drug_target_sequence.head()
d = drug_target_sequence["SubjectID"]
intersect = set(intersect).intersection(d)

In [19]:
import itertools
import numpy as np

def matrix(a, b, match_score=3, gap_cost=2):
    H = np.zeros((len(a) + 1, len(b) + 1), np.int32)

    for i, j in itertools.product(range(1, H.shape[0]), range(1, H.shape[1])):
        match = H[i - 1, j - 1] + (match_score if a[i - 1] == b[j - 1] else - match_score)
        delete = H[i - 1, j] - gap_cost
        insert = H[i, j - 1] - gap_cost
        H[i, j] = max(match, delete, insert, 0)
    return H

In [124]:
def target_seq_similarity(df):
    target_sequence_list = {k: g["seq"].tolist() for k,g in df.groupby("SubjectID")} 
    scores = []

    for combination in itertools.combinations(intersect,2): 
        drug1 = combination[0]
        drug2 = combination[1]
        if not (drug1 in target_sequence_list and drug2 in target_sequence_list) : continue
        target_list1 = target_sequence_list[drug1]
        target_list2 = target_sequence_list[drug2]
        allscores = []
        for target1 in sorted(target_list1): 
            genescores = []
            for target2 in sorted(target_list2): 
                target1 = str(target1)
                target2 = str(target2)
                if target1 == target2:
                    score = 1.0 
                else: 
                    score = np.mean(matrix(target1,target2))
        scores.append([drug1,drug2,score])
    df = pd.DataFrame(scores, columns = ["Drug1", "Drug2", "TARGETSEQ-SIM"])
    return df   

In [29]:
targetseq = target_seq_similarity(drug_target_sequence)

KeyboardInterrupt: 

In [ ]:
targetseq.to_csv("/Users/elifozkan/Desktop/translator-openpredict/data/drkg/feature_related/output/drugs-TARGETSEQ-sim.csv",index=False)

# PPI-Based Similarity from DRKG
### Using the target genes, we first find the protein interactions using the relevant predicates from drkg, then form a graph using the nodes obtained from DRKG. 

In [125]:
ppi_gnbr = GNBR.loc[(GNBR["Predicate"] == "GNBR::B::Gene:Gene") | 
                        (GNBR["Predicate"] == "GNBR::E+::Geme:Gene") |
                         (GNBR["Predicate"] == "GNBR::E::Gene:Gene") |
                          (GNBR["Predicate"] == "GNBR::I::Gene:Gene") |
                           (GNBR["Predicate"] == "GNBR::Rg::Gene:Gene") |
                           (GNBR["Predicate"] == "GNBR::Q::Gene:Gene")|
                           (GNBR["Predicate"] == "GNBR::V+::Gene:Gene")|
                           (GNBR["Predicate"] == "GNBR::W::Gene:Gene")]

ppi_hetionet = Hetionet.loc[Hetionet["Predicate"] == "Hetionet::GIG::Gene:Gene"]

ppi_intact = IntAct.loc[(IntAct["Predicate"] == "INTACT::ASSOCIATION::Gene:Gene" )|
                        (IntAct["Predicate"] == "INTACT::DIRECT INTERACTION::Gene:Gene" ) |
                        (IntAct["Predicate"] == "INTACT::PHYSICAL ASSOCIATION::Gene:Gene" )]

ppi_string = STRING.loc[(STRING["Predicate"] == "STRING::ACTIVATION::Gene:Gene")|
                        (STRING["Predicate"] == "STRING::BINDING::Gene:Gene") |
                        (STRING["Predicate"] == "STRING::CATALYSIS::Gene:Gene")|
                        (STRING["Predicate"] == "STRING::INHIBITION::Gene:Gene")]

In [126]:
ppi = pd.concat([ppi_gnbr,ppi_hetionet,ppi_intact],ignore_index=True)
ppi = ppi[["SubjectID","ObjectID"]]
ppi = ppi.drop_duplicates().reset_index()
ppi

,index,SubjectID,ObjectID
0,0,6187,2248
1,1,563,9370
2,2,6584,5243
3,3,10747,50639
4,5,8573,10716
...,...,...,...
246144,258610,9808,6184
246145,258611,9808,7157
246146,258612,9808,84675
246147,258613,9808,84676


In [127]:
import networkx as nx 
ppi_net = pd.read_csv("/Users/elifozkan/Desktop/translator-openpredict/data/drkg/feature_related/ppi_network.csv")
G = nx.Graph()
for i in range(0,ppi_net.shape[0]): 
    G.add_edge(ppi_net["geneid1"][i],ppi_net["geneid2"][i])
ppi_n=nx.shortest_path_length(G)

In [128]:
def grapDistance(ppi, target1, target2):
    """
    return the shortest path between two proteins in the PPI network
    ppi : dictonary that contains distance of PPI 
    target1 : first protein name
    target2 : second protein name
    """
    maxValue = 9999
    if target1 not in ppi:
        return maxValue
    else:
        if target2 not in ppi[target1]:
            return maxValue
        else:
            return ppi[target1][target2]

In [129]:
import math
target_gene_list = {k: g["ObjectID"].tolist() for k,g in drug_target_seq.groupby("SubjectID")}
values = []

A = 0.9
b = 1
for comb in itertools.combinations(intersect,2) :
    drug1 = comb[0]
    drug2 = comb[1]
    if not(drug1 in target_gene_list and drug2 in target_gene_list) : continue
    targetList1 = target_gene_list[drug1]
    targetList2 = target_gene_list[drug2]
    allscores =[]
    for target1 in sorted(targetList1):
        genescores = []
        for target2 in sorted(targetList2):
            target1 =str(target1)
            target2 =str(target2)    
            if target1 == target2:
                score=1.0
            else:
                score = A*math.exp(-b* grapDistance(ppi_net, target1, target2))
            genescores.append(score)
    # add maximal values between the two lists of associated genes 
    allscores.append(max(genescores))
    if len(allscores) ==0: continue
    #average the maximal scores 
    maxScore =np.mean(allscores)
    if maxScore >= 0:
        values.append([drug1, drug2, maxScore])


In [55]:
drug_ppi_df = pd.DataFrame(values, columns =['Drug1','Drug2','PPI-SIM'])
drug_ppi_df.to_csv("/Users/elifozkan/Desktop/translator-openpredict/data/drkg/feature_related/output/drug-ppi-sim.csv", index=False)

# GO-Based Similarity

In [130]:
drug_go = pd.read_csv("/Users/elifozkan/Desktop/translator-openpredict/data/drkg/feature_related/go-sim.csv")
drug_go["g"] = drug_go["g"].str.replace("go:", "GO_")
drug_go.head()

,drugid,g
0,DB00122,GO_0043083
1,DB00122,GO_0005576
2,DB00122,GO_0005615
3,DB00122,GO_0005634
4,DB00122,GO_0005794


In [41]:
target_gene_list = {k: g["g"].tolist() for k,g in drug_go.groupby("drugid")}
rows = []
for comb in itertools.combinations(drug_go["drugid"].unique(),2):
    drug1 = comb[0]
    drug2 = comb[1]
    rows.append(['http://purl.obolibrary.org/obo/' + str(drug1),'http://purl.obolibrary.org/obo/'+str(drug2)])


drug_query_df = pd.DataFrame(rows, columns =['Drug1','Drug2'])
drug_query_df.to_csv("/Users/elifozkan/Desktop/translator-openpredict/data/drkg/feature_related/drug.gene.go.query",sep='\t', header=False, index=False)
drug_query_df.head()

,Drug1,Drug2
0,http://purl.obolibrary.org/obo/DB00122,http://purl.obolibrary.org/obo/DB00677
1,http://purl.obolibrary.org/obo/DB00122,http://purl.obolibrary.org/obo/DB00119
2,http://purl.obolibrary.org/obo/DB00122,http://purl.obolibrary.org/obo/DB00132
3,http://purl.obolibrary.org/obo/DB00122,http://purl.obolibrary.org/obo/DB00114
4,http://purl.obolibrary.org/obo/DB00122,http://purl.obolibrary.org/obo/DB00116


In [42]:
! if [ ! -f '/Users/elifozkan/Desktop/translator-openpredict/data/drkg/feature_related/go.owl' ]; then gunzip '/Users/elifozkan/Desktop/translator-openpredict/data/drkg/feature_related/go.owl'; fi 

In [43]:
!java -jar /Users/elifozkan/Desktop/sml-toolkit-0.9.4c.jar -t sm -xmlconf /Users/elifozkan/Desktop/translator-openpredict/data/drkg/feature_related/sml.gene.go.conf

----------------------------------------------------------------------
	Semantic Measures Library Toolkit 0.9.4c
----------------------------------------------------------------------
Toolkit dedicated to Semantic Measures computation and studies.
Website: http://www.semantic-measures-library.org
Developer: Sébastien Harispe

Please cite: 
The Semantic Measures Library and Toolkit: 
fast computation of semantic similarity and relatedness 
using biomedical ontologies.
Sébastien Harispe*, Sylvie Ranwez, Stefan Janaqi, Jacky Montmain
Bioinformatics 2013; doi: 10.1093/bioinformatics/btt581
----------------------------------------------------------------------
Args [-t, sm]
Args Tool    [-xmlconf, /Users/elifozkan/Desktop/translator-openpredict/data/drkg/feature_related/sml.gene.go.conf]
Loading SM Tool
----------------------------------------------------------------------
	Semantic Measures Computer 0.9.4c
----------------------------------------------------------------------
Tool used to 

In [39]:
go_sim_df = pd.read_csv("/Users/elifozkan/Desktop/translator-openpredict/data/drkg/feature_related/drug.gene.go.sim.out",sep='\t')
go_sim_df.rename(columns={'e1':'Drug1','e2':'Drug2','bma':'GO-SIM'}, inplace=True)
go_sim_df.Drug1 = go_sim_df.Drug1.str.replace('http://purl.obolibrary.org/obo/','')
go_sim_df.Drug2 = go_sim_df.Drug2.str.replace('http://purl.obolibrary.org/obo/','')
go_sim_df.to_csv("/Users/elifozkan/Desktop/translator-openpredict/data/drkg/feature_related/output/drug-go-sim.csv",index=False)

/var/folders/v7/vrxjycqx6736th34cms70szm0000gn/T/ipykernel_2357/1305721123.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  go_sim_df.Drug1 = go_sim_df.Drug1.str.replace('http://purl.obolibrary.org/obo/','')
/var/folders/v7/vrxjycqx6736th34cms70szm0000gn/T/ipykernel_2357/1305721123.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  go_sim_df.Drug2 = go_sim_df.Drug2.str.replace('http://purl.obolibrary.org/obo/','')


# MESH Similarity for Diseases from DRKG

In [20]:
GNBR_diseases = GNBR.loc[GNBR["SubjectType"] == "Disease"]
Hetionet_diseases = Hetionet.loc[Hetionet["SubjectType"] == "Disease"]
DrugBank_diseases = DrugBank.loc[DrugBank["SubjectType"] == "Disease"]

In [21]:
diseases = pd.concat([GNBR_diseases,Hetionet_diseases,DrugBank_diseases])
diseases

,Subject,Predicate,Object,Source1,Source2,Source3,Source4,Source5,Source6,SubjectType,SubjectID,ObjectType,ObjectID
4510005,Disease::MESH:D000506,Hetionet::DdG::Disease:Gene,Gene::26059,[Drugbank],[Hetionet],[GNBR],NaN,NaN,NaN,Disease,MESH:D000506,Gene,26059
4510006,Disease::MESH:D000506,Hetionet::DdG::Disease:Gene,Gene::84445,[Drugbank],[Hetionet],[GNBR],NaN,NaN,NaN,Disease,MESH:D000506,Gene,84445
4510007,Disease::MESH:D000506,Hetionet::DdG::Disease:Gene,Gene::9632,[Drugbank],[Hetionet],[GNBR],NaN,NaN,NaN,Disease,MESH:D000506,Gene,9632
4510008,Disease::MESH:D000506,Hetionet::DdG::Disease:Gene,Gene::8774,[Drugbank],[Hetionet],[GNBR],NaN,NaN,NaN,Disease,MESH:D000506,Gene,8774
4510009,Disease::MESH:D000506,Hetionet::DdG::Disease:Gene,Gene::131578,[Drugbank],[Hetionet],[GNBR],NaN,NaN,NaN,Disease,MESH:D000506,Gene,131578
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4545465,Disease::MESH:D005923,Hetionet::DaG::Disease:Gene,Gene::81,[Drugbank],[Hetionet],[GNBR],NaN,NaN,NaN,Disease,MESH:D005923,Gene,81
4545466,Disease::MESH:D005923,Hetionet::DaG::Disease:Gene,Gene::7436,[Drugbank],[Hetionet],[GNBR],NaN,NaN,NaN,Disease,MESH:D005923,Gene,7436
4545467,Disease::MESH:D005923,Hetionet::DaG::Disease:Gene,Gene::1286,[Drugbank],[Hetionet],[GNBR],NaN,NaN,NaN,Disease,MESH:D005923,Gene,1286
4545468,Disease::MESH:D005923,Hetionet::DaG::Disease:Gene,Gene::5054,[Drugbank],[Hetionet],[GNBR],NaN,NaN,NaN,Disease,MESH:D005923,Gene,5054


In [22]:
mesh = pd.read_csv("/Users/elifozkan/Desktop/translator-openpredict/data/drkg/feature_related/disease_terms.csv")
mesh["SubjectID"]= mesh["SubjectID"].str.upper()
mesh_diseases = pd.merge(mesh,diseases,on = "SubjectID")
mesh_diseases = mesh_diseases[["SubjectID","Term"]]
mesh_diseases = mesh_diseases.drop_duplicates()

In [23]:
disease_mesh_dict = {k : g["Term"].tolist() for k,g in mesh_diseases.groupby("SubjectID")}
sim_scores = []

for combination in itertools.combinations(mesh_diseases["SubjectID"].unique(),2):
    disease1 = combination[0]
    disease2 = combination[1]

    term1 = disease_mesh_dict[disease1]
    term2 = disease_mesh_dict[disease2]

    c = set(term1).intersection(term2)
    u = set(term2).union(term2)

    sim_score = len(c) / float(len(u))
    sim_scores.append([disease1,disease2,sim_score])
disease_pheno_df = pd.DataFrame(sim_scores, columns = ["Disease1", "Disease2", "PHENO-SIM"])


In [24]:
disease_pheno_df.to_csv("/Users/elifozkan/Desktop/translator-openpredict/data/drkg/feature_related/output/disease-pheno.csv",index=False)

In [37]:
genes_to_pheno = pd.read_csv("/Users/elifozkan/Desktop/translator-openpredict/data/drkg/feature_related/genes_to_phenotype.tsv",on_bad_lines="skip")

In [38]:
genes_to_pheno["HPO-Term-Name"] = genes_to_pheno["HPO-Term-Name"].str.lower()
genes_to_pheno = genes_to_pheno.rename(columns = {"HPO-Term-Name" : "name"})
genes_to_pheno = genes_to_pheno[["name", "disease-ID for link", "HPO-Term-ID"]]
genes_to_pheno

,name,disease-ID for link,HPO-Term-ID
0,hypoplasia of the uterus,OMIM:614129,HP:0000013
1,hypergonadotropic hypogonadism,OMIM:614129,HP:0000815
2,primary amenorrhea,OMIM:614129,HP:0000786
3,autosomal recessive inheritance,OMIM:614129,HP:0000007
4,microcephaly,OMIM:614129,HP:0000252
...,...,...,...
268978,functional motor deficit,ORPHA:98934,HP:0004302
268979,personality changes,ORPHA:98934,HP:0000751
268980,memory impairment,ORPHA:98934,HP:0002354
268981,dementia,ORPHA:98934,HP:0000726


In [39]:
mesh_diseases_vocab = pd.read_csv("/Users/elifozkan/Desktop/translator-openpredict/data/drkg/feature_related/MESH_diseases_vocab.tsv", sep = "\t")
mesh_diseases_vocab["name"] = mesh_diseases_vocab["name"].str.lower()
mesh_diseases_vocab

,d,name
0,mesh:D053559,"corneal dystrophy, juvenile epithelial of mees..."
1,mesh:D015715,corneal edema
2,mesh:D055954,corneal endothelial cell loss
3,mesh:D065306,corneal injuries
4,mesh:D016510,corneal neovascularization
...,...,...
4995,mesh:D008363,alpha-mannosidosis
4996,mesh:D017085,alpha-thalassemia
4997,mesh:D044905,beta-mannosidosis
4998,mesh:D017086,beta-thalassemia


In [43]:
mappings = pd.merge(mesh_diseases_vocab,genes_to_pheno,on = "name")
mappings["d"] = mappings["d"].str.upper()
mappings = mappings.rename(columns={"d" : "SubjectID"})
mappings

,SubjectID,name,disease-ID for link,HPO-Term-ID
0,MESH:D016510,corneal neovascularization,ORPHA:567,HP:0011496
1,MESH:D016510,corneal neovascularization,OMIM:175780,HP:0011496
2,MESH:D016510,corneal neovascularization,ORPHA:567,HP:0011496
3,MESH:D016510,corneal neovascularization,ORPHA:567,HP:0011496
4,MESH:D016510,corneal neovascularization,OMIM:617315,HP:0011496
...,...,...,...,...
59075,MESH:D015043,zollinger-ellison syndrome,ORPHA:97279,HP:0002044
59076,MESH:D015043,zollinger-ellison syndrome,ORPHA:652,HP:0002044
59077,MESH:D015043,zollinger-ellison syndrome,OMIM:131100,HP:0002044
59078,MESH:D015043,zollinger-ellison syndrome,OMIM:248700,HP:0002044


In [46]:
disease_to_hpo = pd.merge(mappings,diseases, on = "SubjectID")
disease_to_hpo = disease_to_hpo[["SubjectID","HPO-Term-ID"]]
disease_to_hpo

,SubjectID,HPO-Term-ID
0,MESH:D005901,HP:0000501
1,MESH:D005901,HP:0000501
2,MESH:D005901,HP:0000501
3,MESH:D005901,HP:0000501
4,MESH:D005901,HP:0000501
...,...,...
3284778,MESH:D014820,HP:0001045
3284779,MESH:D014820,HP:0001045
3284780,MESH:D014820,HP:0001045
3284781,MESH:D014820,HP:0001045


In [ ]:
disease_to_hpo["SubjectID"]=disease_to_hpo["SubjectID"].map(lambda d: d)

In [62]:
disease_to_hpo = disease_to_hpo.drop_duplicates()

In [63]:

disease_to_hpo.to_csv("/Users/elifozkan/Desktop/translator-openpredict/data/drkg/feature_related/disease-hpo.query",sep = "\t", header = False, index=False)

In [64]:

disease_to_hpo.to_csv("/Users/elifozkan/Desktop/translator-openpredict/data/drkg/feature_related/disease-hpo.tsv",sep = "\t", header = False, index=False)

In [65]:
! if [ ! -f '/Users/elifozkan/Desktop/translator-openpredict/data/drkg/feature_related/hp.owl' ]; then gunzip '/Users/elifozkan/Desktop/translator-openpredict/data/drkg/feature_related/hp.owl'; fi 

In [66]:
!java -jar /Users/elifozkan/Desktop/sml-toolkit-0.9.4c.jar -t sm -xmlconf /Users/elifozkan/Desktop/translator-openpredict/data/drkg/feature_related/hpo.conf

----------------------------------------------------------------------
	Semantic Measures Library Toolkit 0.9.4c
----------------------------------------------------------------------
Toolkit dedicated to Semantic Measures computation and studies.
Website: http://www.semantic-measures-library.org
Developer: Sébastien Harispe

Please cite: 
The Semantic Measures Library and Toolkit: 
fast computation of semantic similarity and relatedness 
using biomedical ontologies.
Sébastien Harispe*, Sylvie Ranwez, Stefan Janaqi, Jacky Montmain
Bioinformatics 2013; doi: 10.1093/bioinformatics/btt581
----------------------------------------------------------------------
Args [-t, sm]
Args Tool    [-xmlconf, /Users/elifozkan/Desktop/translator-openpredict/data/drkg/feature_related/hpo.conf]
Loading SM Tool
----------------------------------------------------------------------
	Semantic Measures Computer 0.9.4c
----------------------------------------------------------------------
Tool used to compute 

In [69]:
diseases

,Subject,Predicate,Object,Source1,Source2,Source3,Source4,Source5,Source6,SubjectType,SubjectID,ObjectType,ObjectID
4510005,Disease::MESH:D000506,Hetionet::DdG::Disease:Gene,Gene::26059,[Drugbank],[Hetionet],[GNBR],NaN,NaN,NaN,Disease,MESH:D000506,Gene,26059
4510006,Disease::MESH:D000506,Hetionet::DdG::Disease:Gene,Gene::84445,[Drugbank],[Hetionet],[GNBR],NaN,NaN,NaN,Disease,MESH:D000506,Gene,84445
4510007,Disease::MESH:D000506,Hetionet::DdG::Disease:Gene,Gene::9632,[Drugbank],[Hetionet],[GNBR],NaN,NaN,NaN,Disease,MESH:D000506,Gene,9632
4510008,Disease::MESH:D000506,Hetionet::DdG::Disease:Gene,Gene::8774,[Drugbank],[Hetionet],[GNBR],NaN,NaN,NaN,Disease,MESH:D000506,Gene,8774
4510009,Disease::MESH:D000506,Hetionet::DdG::Disease:Gene,Gene::131578,[Drugbank],[Hetionet],[GNBR],NaN,NaN,NaN,Disease,MESH:D000506,Gene,131578
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4545465,Disease::MESH:D005923,Hetionet::DaG::Disease:Gene,Gene::81,[Drugbank],[Hetionet],[GNBR],NaN,NaN,NaN,Disease,MESH:D005923,Gene,81
4545466,Disease::MESH:D005923,Hetionet::DaG::Disease:Gene,Gene::7436,[Drugbank],[Hetionet],[GNBR],NaN,NaN,NaN,Disease,MESH:D005923,Gene,7436
4545467,Disease::MESH:D005923,Hetionet::DaG::Disease:Gene,Gene::1286,[Drugbank],[Hetionet],[GNBR],NaN,NaN,NaN,Disease,MESH:D005923,Gene,1286
4545468,Disease::MESH:D005923,Hetionet::DaG::Disease:Gene,Gene::5054,[Drugbank],[Hetionet],[GNBR],NaN,NaN,NaN,Disease,MESH:D005923,Gene,5054


In [73]:
disease_to_hpo.to_csv("/Users/elifozkan/Desktop/translator-openpredict/data/drkg/feature_related/output/disease-hpo.csv", index = False)